In [ ]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

In [ ]:
import os
# Office
os.chdir('C:\\Users\\prudi\\Documents\\GitHub\\Code\\NITDATA\\Algorithms\\codesnippetsforbaggingandboostingalgorithms')
data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

class PreProcessing(BaseEstimator, TransformerMixin):


    def __init__(self):
        pass

    
    ## Here this model will be called with a variable (df), which is a dataframe
    def datapreprocessing(self, df):
        
        df = df.reset_index(drop=True)
        
        df.loc[df['Gender'].isnull(),'Gender']='Trasgender'
        df.loc[df['Dependents'].isnull(),'Dependents']='0'
        df.loc[df['Education'].isnull(),'Education']='No Education'
        df.loc[df['Married'].isnull(),'Married']='No'
        df.loc[df['Self_Employed'].isnull(),'Self_Employed']='No'
        df.loc[df['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
        df.loc[df['Credit_History'].isnull(),'Credit_History']=0
        df.loc[df['LoanAmount'].isnull(),'LoanAmount']=146.867     
        df['Credit_History']=df['Credit_History'].astype(str)
        
        
        ## Creating a Instance for the minmax Pickle File
        MinMaxpickle_in = open("minmax_pickle.pkl","rb")
        MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        
        ## Applying the pickle file
        df['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].transform(df[['ApplicantIncome']]))
        
        ## Creating a Instance for the minmax Pickle File
        standardscaler_in = open("standardscaler_pickle.pkl","rb")
        standardscaler_dict = pickle.load(standardscaler_in)
        
        ## Applying the pickle file
        df['LoanAmount']=pd.DataFrame(standardscaler_dict['LoanAmount'].transform(df[['LoanAmount']]))
        
        ## Creating a Instance for the LabelEncoder Pickle File
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Education']=pd.DataFrame(Labelencoder_dict['Education'].transform(df[['Education']]))
        df['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].transform(df[['Property_Area']]))
        df['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].transform(df[['Credit_History']]))
        df['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].transform(df[['Dependents']]))
        
        ## Creating a Instance for the binarizer Pickle File
        Onehotpickle_in = open("binarizer_pickle.pkl","rb")
        Onehot_dict = pickle.load(Onehotpickle_in)
        
        ## Applying the pickle file
        d1=pd.DataFrame(Onehot_dict['Self_Employed'].transform(df[['Self_Employed']]))
        d1.columns=['Self_Employed_0']
        d2=pd.DataFrame(Onehot_dict['Married'].transform(df[['Married']]))
        d2.columns=['Married_0']
        d3=pd.DataFrame(Onehot_dict['Gender'].transform(df[['Gender']]))
        d3.columns=['Gender_0','Gender_1','Gender_2']
    
        df=df.drop(['Self_Employed','Married','Gender','Loan_ID'],axis=1)
        df=pd.concat([df,d1,d2,d3],axis=1)
        
        return df

    def definingvalues(self, df, y=None,**fit_params):
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self
    
    def encodingTargetVariable(self,df):
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Loan_Status']=pd.DataFrame(Labelencoder_dict['Loan_Status'].transform(df[['Loan_Status']]))
        
        return df.as_matrix()

In [ ]:
data.columns

In [ ]:
pred_var = ['Loan_ID','Gender','Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']
X_train, X_test, y_train, y_test = train_test_split(data[pred_var], data['Loan_Status'], test_size=0.30, random_state=42)

In [ ]:
## Creating an instance for the class
preprocess = PreProcessing()

In [ ]:
# Calling the fit function present in the class Preprocessing
preprocess.definingvalues(X_train)

In [ ]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_train dataset
data_transformed_xtrain = preprocess.datapreprocessing(X_train)

pred_var = ['Dependents', 'Education', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area', 'Self_Employed_0', 'Married_0', 'Gender_0', 'Gender_1','Gender_2']


data_transformed_xtrain=pd.DataFrame(data_transformed_xtrain,columns=pred_var)
data_transformed_xtrain.head()

print(data_transformed_xtrain.shape)
print(X_train.shape)

In [ ]:
data_transformed_xtrain.columns

In [ ]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_test dataset
data_transformed_xtest = preprocess.datapreprocessing(X_test)

# Converting the matrix to a dataframe
data_transformed_xtest=pd.DataFrame(data_transformed_xtest,columns=pred_var)
data_transformed_xtest.head()

In [ ]:
y_train=pd.DataFrame(y_train).reset_index(drop=True)
y_train.columns=['Loan_Status']

y_test=pd.DataFrame(y_test).reset_index(drop=True)
y_test.columns=['Loan_Status']

y_train=preprocess.encodingTargetVariable(y_train)
y_test=preprocess.encodingTargetVariable(y_test)

In [ ]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

import os
# Office
os.chdir('C:\\Users\\prudi\\Documents\\GitHub\\Code\\NITDATA\\Algorithms\\codesnippetsforbaggingandboostingalgorithms')
data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

from sklearn.base import BaseEstimator, TransformerMixin
import pickle

class PreProcessing(BaseEstimator, TransformerMixin):


    def __init__(self):
        pass

    
    ## Here this model will be called with a variable (df), which is a dataframe
    def datapreprocessing(self, df):
        
        df = df.reset_index(drop=True)
        
        df.loc[df['Gender'].isnull(),'Gender']='Trasgender'
        df.loc[df['Dependents'].isnull(),'Dependents']='0'
        df.loc[df['Education'].isnull(),'Education']='No Education'
        df.loc[df['Married'].isnull(),'Married']='No'
        df.loc[df['Self_Employed'].isnull(),'Self_Employed']='No'
        df.loc[df['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
        df.loc[df['Credit_History'].isnull(),'Credit_History']=0
        df.loc[df['LoanAmount'].isnull(),'LoanAmount']=146.867     
        df['Credit_History']=df['Credit_History'].astype(str)
        
        
        ## Creating a Instance for the minmax Pickle File
        MinMaxpickle_in = open("minmax_pickle.pkl","rb")
        MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        
        ## Applying the pickle file
        df['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].transform(df[['ApplicantIncome']]))
        
        ## Creating a Instance for the minmax Pickle File
        standardscaler_in = open("standardscaler_pickle.pkl","rb")
        standardscaler_dict = pickle.load(standardscaler_in)
        
        ## Applying the pickle file
        df['LoanAmount']=pd.DataFrame(standardscaler_dict['LoanAmount'].transform(df[['LoanAmount']]))
        
        ## Creating a Instance for the LabelEncoder Pickle File
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Education']=pd.DataFrame(Labelencoder_dict['Education'].transform(df[['Education']]))
        df['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].transform(df[['Property_Area']]))
        df['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].transform(df[['Credit_History']]))
        df['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].transform(df[['Dependents']]))
        
        ## Creating a Instance for the binarizer Pickle File
        Onehotpickle_in = open("binarizer_pickle.pkl","rb")
        Onehot_dict = pickle.load(Onehotpickle_in)
        
        ## Applying the pickle file
        d1=pd.DataFrame(Onehot_dict['Self_Employed'].transform(df[['Self_Employed']]))
        d1.columns=['Self_Employed_0']
        d2=pd.DataFrame(Onehot_dict['Married'].transform(df[['Married']]))
        d2.columns=['Married_0']
        d3=pd.DataFrame(Onehot_dict['Gender'].transform(df[['Gender']]))
        d3.columns=['Gender_0','Gender_1','Gender_2']
    
        df=df.drop(['Self_Employed','Married','Gender','Loan_ID'],axis=1)
        df=pd.concat([df,d1,d2,d3],axis=1)
        
        return df

    def definingvalues(self, df, y=None,**fit_params):
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self
    
    def encodingTargetVariable(self,df):
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Loan_Status']=pd.DataFrame(Labelencoder_dict['Loan_Status'].transform(df[['Loan_Status']]))
        
        return df.as_matrix()

data.columns

pred_var = ['Loan_ID','Gender','Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']
X_train, X_test, y_train, y_test = train_test_split(data[pred_var], data['Loan_Status'], test_size=0.30, random_state=42)

## Creating an instance for the class
preprocess = PreProcessing()

# Calling the fit function present in the class Preprocessing
preprocess.definingvalues(X_train)

## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_train dataset
data_transformed_xtrain = preprocess.datapreprocessing(X_train)

pred_var = ['Dependents', 'Education', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area', 'Self_Employed_0', 'Married_0', 'Gender_0', 'Gender_1','Gender_2']


data_transformed_xtrain=pd.DataFrame(data_transformed_xtrain,columns=pred_var)
data_transformed_xtrain.head()

print(data_transformed_xtrain.shape)
print(X_train.shape)

data_transformed_xtrain.columns

## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_test dataset
data_transformed_xtest = preprocess.datapreprocessing(X_test)

# Converting the matrix to a dataframe
data_transformed_xtest=pd.DataFrame(data_transformed_xtest,columns=pred_var)
data_transformed_xtest.head()

y_train=pd.DataFrame(y_train).reset_index(drop=True)
y_train.columns=['Loan_Status']

y_test=pd.DataFrame(y_test).reset_index(drop=True)
y_test.columns=['Loan_Status']

y_train=preprocess.encodingTargetVariable(y_train)
y_test=preprocess.encodingTargetVariable(y_test)



### Applying BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
BagCla = BaggingClassifier(base_estimator=LogisticRegression(C=10,penalty='l2'),n_estimators=10, 
                           random_state=0,max_features=0.5,max_samples=0.8)
BagCla.fit(data_transformed_xtrain, y_train)

In [ ]:
BagCla.predict(data_transformed_xtrain)

In [ ]:
BagCla.predict_proba(data_transformed_xtrain)